In [22]:
from pythainlp.tokenize import word_tokenize
import torch
from torch import nn
import torch.optim as optim
import pandas as pd
import deepcut

In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thaweewat-data. Use `wandb login --relogin` to force relogin


In [38]:
train = (pd.read_csv(r"C:\Users\rthaw\Downloads\train.csv", header=None)
		 .rename(columns={0:'comment', 1:'label'})
		 .assign(text=lambda x: x['comment'].str.replace(r'[^ก-๙0-9]','',regex=True))
		 .query('text != ""')
		 )

train.head()

,comment,label,text
0,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ
1,คะ,neu,คะ
2,อิเหี้ยออมทำกูอยากกินเอ็มเค,neg,อิเหี้ยออมทำกูอยากกินเอ็มเค
4,สวัสดีวันพุธ แนน อะไรนะ,neu,สวัสดีวันพุธแนนอะไรนะ
5,ก้อนขอบพระคุณมากๆนะคร้าบ ที่มาหาก้อนและชมเชยกา...,neu,ก้อนขอบพระคุณมากๆนะคร้าบที่มาหาก้อนและชมเชยการ...


In [47]:
train

,comment,label,text
0,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ
1,คะ,neu,คะ
2,อิเหี้ยออมทำกูอยากกินเอ็มเค,neg,อิเหี้ยออมทำกูอยากกินเอ็มเค
4,สวัสดีวันพุธ แนน อะไรนะ,neu,สวัสดีวันพุธแนนอะไรนะ
5,ก้อนขอบพระคุณมากๆนะคร้าบ ที่มาหาก้อนและชมเชยกา...,neu,ก้อนขอบพระคุณมากๆนะคร้าบที่มาหาก้อนและชมเชยการ...
...,...,...,...
24058,แม่งควายล้วนนน,neg,แม่งควายล้วนนน
24059,ดอยสุเทพน้องง ไปหมดแล้วววว #pm25,neg,ดอยสุเทพน้องงไปหมดแล้วววว25
24060,ค่าชุดอาจจะแพงกว่าส่วนลด,neg,ค่าชุดอาจจะแพงกว่าส่วนลด
24061,รัฐต้องการแค่ภาษีครับ,neu,รัฐต้องการแค่ภาษีครับ


In [62]:
sentences = train.comment.values
sentences

array(['ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ', 'คะ',
       'อิเหี้ยออมทำกูอยากกินเอ็มเค', ..., 'ค่าชุดอาจจะแพงกว่าส่วนลด',
       'รัฐต้องการแค่ภาษีครับ',
       '™ ได้พัฒนาระบบเข้าสู่รุ่นที่สองแล้ว มาดูกันสิว่า เจ้า TSS จะฉลาดขึ้นอย่างไรกันบ้างค่ะ'],
      dtype=object)

In [3]:
corpus = open('datasets/corpus.txt', 'r', encoding='utf-8').readlines()

In [51]:
words = []
for sentences in sentences:
	tokens = word_tokenize(sentences.strip())
	for word in tokens:
		words.append(word)

# print the first 10 words


In [52]:
# coutn words
from collections import Counter

word_counts = Counter(words)
print(f"Number of unique words : {len(word_counts)}")

Number of unique words : 31066


In [45]:
words = []
for sentences in sentences:
	tokens = deepcut.tokenize(sentences.strip())
	for word in tokens:
		words.append(word)

print(words[:10])

2/2 [==============================] - 0s 5ms/step



KeyboardInterrupt



In [54]:
wandb.init(project='nlp_class', entity="thaweewat-data", name='word2vec_wisesight')

class Word2Vec(nn.Module):
	def __init__(self, vocab_size, embed_size):
		super(Word2Vec, self).__init__()
		self.embed = nn.Embedding(vocab_size, embed_size)

	def forward(self, x):
		x = self.embed(x)
		return x

# get unique words
unique_words = list(set(words))

# create a word to index mapping
word_to_ix = {word: i for i, word in enumerate(unique_words)}
ix_to_word = {i: word for i, word in enumerate(unique_words)}

# dummy data
pairs = [(word_to_ix[words[i]], word_to_ix[words[i+1]]) for i in range(len(words)-1)]

# Initialize the model and the optimizer
model = Word2Vec(vocab_size=len(word_to_ix), embed_size=100)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convert the pairs to tensors
pairs = [(torch.tensor(pair[0]), torch.tensor(pair[1])) for pair in pairs]

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
	total_loss = 0
	for context, target in pairs[:-1]:  # Exclude the last pair
		context_tensor = context.unsqueeze(0)  # add an extra dimension for batch
		target_tensor = target.unsqueeze(0)  # add an extra dimension for batch

		# Forward pass
		outputs = model(context_tensor)
		target_embedding = model(target_tensor)
		loss = nn.CosineEmbeddingLoss()(outputs.squeeze(0), target_embedding.squeeze(0), torch.tensor(1.0))

		# Backward pass and optimization
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		total_loss += loss.item()

	wandb.log({'epoch': epoch+1, 'loss': total_loss/len(pairs[:-1])})  # Modify the denominator here as well

# save model
torch.save(model.state_dict(), 'word2vec.pth')

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,1000
loss,0.00039



KeyboardInterrupt



In [18]:
# use the trained model
word = 'ท้องฟ้า'
word_idx = torch.tensor([word_to_ix[word]])

word_vector = model(word_idx)
print(f"Vector representation of {word} : {word_vector}")

Vector representation of ท้องฟ้า : tensor([[ 0.5607, -0.7273,  1.9554,  1.5201,  5.6732, -1.0722,  2.3450,  0.5684,
         -1.1541,  0.5305, -2.1501, -0.8991, -0.6944,  0.5944,  4.1595, -4.2660,
          0.1220,  1.3683,  1.4392,  1.6408, -0.2084,  0.1102, -0.0572, -0.3220,
          2.2858,  0.4622, -1.9974,  4.3646, -0.3536,  2.9242,  1.9639, -0.4516,
         -0.7355,  0.8392, -2.0328, -0.2936,  0.8000,  2.1336,  4.8581,  1.3202,
          2.7935,  1.2710, -1.8408, -1.5237,  1.8223, -0.3685, -3.6957, -1.8174,
         -0.6285, -2.3897, -0.9761, -1.0413,  4.0978,  0.3500, -2.1603, -0.1558,
          0.1421, -6.4632, -3.4550,  3.5997,  1.3545,  4.7173, -1.4564,  3.7883,
          0.6037, -2.6363, -1.1228, -0.3013, -1.3972,  4.6163,  1.1017,  1.4028,
         -0.0407, -2.3284, -2.1800, -1.9439,  2.2524,  2.2509, -1.4748, -2.7982,
          2.2439, -1.2240,  2.3132, -0.7525,  0.9704,  2.5508, -0.9856,  2.1092,
          0.9523,  0.8730, -2.8033,  0.6003,  0.1363,  3.7563,  1.7361,  1

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

word = 'ที่'
word_idx = torch.tensor([word_to_ix[word]])
word_vector = model(word_idx)
similarity_scores = cosine_similarity(word_vector.detach().numpy(), model.embed.weight.detach().numpy())
similar_words = np.argsort(similarity_scores, axis=1)[0][-5:]

print(f"Top 5 similar words to {word} : {[ix_to_word[idx] for idx in similar_words]}")

Top 5 similar words to ที่ : ['ท้องฟ้า', 'ชวน', 'กระแสลม', 'มัน', 'ที่']


In [17]:
# get the word vectors
word_vectors = model.embed.weight.detach().numpy()

# create a dataframe
df = pd.DataFrame(word_vectors, index=ix_to_word.values())
df.to_csv('w2v.tsv', sep='\t', header=False)

with open('label.tsv', 'w', encoding='utf-8') as f:
	for word in ix_to_word.values():
		f.write(f"{word}\n")

In [55]:
import numpy as np

from gensim.models import Word2Vec

In [63]:
tsentences = []
for st in sentences:
	tokens = word_tokenize(st.strip())
	tsentences.append(tokens)

tsentences[0:5]

[['ประเทศ',
  'เรา',
  'ผลิต',
  'และ',
  'ส่งออก',
  'ยาสูบ',
  'เยอะ',
  'สุด',
  'ใน',
  'โลก',
  'จิง',
  'ป่าว',
  'คับ'],
 ['คะ'],
 ['อิ', 'เหี้ย', 'ออม', 'ทำ', 'กู', 'อยาก', 'กิน', 'เอ็ม', 'เค'],
 ['สวัสดี', 'วัน', 'พุธ', ' ', 'แนน', ' ', 'อะไร', 'นะ'],
 ['ก้อน',
  'ขอบพระคุณ',
  'มาก',
  'ๆ',
  'นะ',
  'คร้าบ',
  ' ',
  'ที่',
  'มาหา',
  'ก้อน',
  'และ',
  'ชมเชย',
  'การ',
  'บริการ',
  'ของ',
  'พนักงาน',
  'อีกด้วย',
  ' ',
  '<',
  '3',
  ' ',
  '<',
  '3']]

In [64]:
%%time
dimension = 128
w2v_model = Word2Vec(tsentences, min_count=1, vector_size=dimension, workers=10, sg=1, epochs=100)

# Get the word embeddings
embeddings = w2v_model.wv.vectors
print(w2v_model)

Word2Vec<vocab=31066, vector_size=128, alpha=0.025>


In [65]:
w2v_model.wv.similarity('อร่อย', 'เยี่ยม')

0.11185953

In [67]:
# get top sim "อร่อย"
w2v_model.wv.most_similar('อร่อย')

[('รสชาติ', 0.5738478302955627),
 ('แปร่ง', 0.5731297731399536),
 ('ชอต', 0.569290041923523),
 ('👍🏼😉🎶', 0.561396062374115),
 ('ๆๆๆๆๆฟๆ', 0.5600540637969971),
 ('หย่อย', 0.5560547709465027),
 ('ถูกปาก', 0.5532785058021545),
 ('ร้ก', 0.5528666973114014),
 ('Jian', 0.5505960583686829),
 ('batteri', 0.5451410412788391)]

In [71]:
# get the word vectors save to tsv files
word_vectors = w2v_model.wv.vectors

# create a dataframe
df = pd.DataFrame(word_vectors, index=w2v_model.wv.index_to_key)
df.to_csv('w2v_gensim.tsv', sep='\t', header=False, encoding='utf-8')

with open('label_gensim.tsv', 'w', encoding='utf-8') as f:
	for word in w2v_model.wv.index_to_key:
		f.write(f"{word}\n")

In [73]:
import polars as pl

In [189]:
data_raw = pl.read_parquet(r"C:\Users\rthaw\Downloads\result_fanboi_sex.parquet")
data_raw

reply_no,reply_name,date,id,reply_text,post,source,source_no
i64,str,str,str,str,str,str,i64
1,"""Nameless Fanbo…","""Posted Ju…","""ID:yR/29ngETb""","""https://fanboi…","""โม่งมิตรสหายท่…","""https://fanboi…",1
2,"""Nameless Fanbo…","""Posted Ju…","""ID:yR/29ngETb""","""ดูเหมือนเหตุผล…","""โม่งมิตรสหายท่…","""https://fanboi…",1
3,"""Nameless Fanbo…","""Posted Ju…","""ID:8ZNLAzChG2""","""ผมเห็นใจเพื่อไ…","""โม่งมิตรสหายท่…","""https://fanboi…",1
4,"""Nameless Fanbo…","""Posted Ju…","""ID:Kne4WM7.ys""","""ฉันเกิดในรัฐธร…","""โม่งมิตรสหายท่…","""https://fanboi…",1
5,"""Nameless Fanbo…","""Posted Ju…","""ID6:4dhpMrXc7N…","""ประชาธิปไตยที่…","""โม่งมิตรสหายท่…","""https://fanboi…",1
6,"""Nameless Fanbo…","""Posted Ju…","""ID:1tfS8OFNGF""","""แดงมึงจะดีใจกั…","""โม่งมิตรสหายท่…","""https://fanboi…",1
7,"""Nameless Fanbo…","""Posted Ju…","""ID6:4dhpMrXc7N…",""">>6 รอดเพราะดี…","""โม่งมิตรสหายท่…","""https://fanboi…",1
8,"""Nameless Fanbo…","""Posted Ju…","""ID6:0InH/af4GP…",""">>7 ก็เห็นดีลไ…","""โม่งมิตรสหายท่…","""https://fanboi…",1
9,"""Nameless Fanbo…","""Posted Ju…","""ID:Zy+T3rcFbz""","""ผลการลงมติของร…","""โม่งมิตรสหายท่…","""https://fanboi…",1


In [190]:
len(data_raw)

4241

In [191]:
data_selected = data_raw
data_selected

reply_no,reply_name,date,id,reply_text,post,source,source_no
i64,str,str,str,str,str,str,i64
1,"""Nameless Fanbo…","""Posted Ju…","""ID:yR/29ngETb""","""https://fanboi…","""โม่งมิตรสหายท่…","""https://fanboi…",1
2,"""Nameless Fanbo…","""Posted Ju…","""ID:yR/29ngETb""","""ดูเหมือนเหตุผล…","""โม่งมิตรสหายท่…","""https://fanboi…",1
3,"""Nameless Fanbo…","""Posted Ju…","""ID:8ZNLAzChG2""","""ผมเห็นใจเพื่อไ…","""โม่งมิตรสหายท่…","""https://fanboi…",1
4,"""Nameless Fanbo…","""Posted Ju…","""ID:Kne4WM7.ys""","""ฉันเกิดในรัฐธร…","""โม่งมิตรสหายท่…","""https://fanboi…",1
5,"""Nameless Fanbo…","""Posted Ju…","""ID6:4dhpMrXc7N…","""ประชาธิปไตยที่…","""โม่งมิตรสหายท่…","""https://fanboi…",1
6,"""Nameless Fanbo…","""Posted Ju…","""ID:1tfS8OFNGF""","""แดงมึงจะดีใจกั…","""โม่งมิตรสหายท่…","""https://fanboi…",1
7,"""Nameless Fanbo…","""Posted Ju…","""ID6:4dhpMrXc7N…",""">>6 รอดเพราะดี…","""โม่งมิตรสหายท่…","""https://fanboi…",1
8,"""Nameless Fanbo…","""Posted Ju…","""ID6:0InH/af4GP…",""">>7 ก็เห็นดีลไ…","""โม่งมิตรสหายท่…","""https://fanboi…",1
9,"""Nameless Fanbo…","""Posted Ju…","""ID:Zy+T3rcFbz""","""ผลการลงมติของร…","""โม่งมิตรสหายท่…","""https://fanboi…",1


In [192]:
dirty_text = data_selected.to_pandas().reply_text

# remove ">>" with number
dirty_text = dirty_text.str.replace(r'>>\d+', '')
# remove ">>"
dirty_text = dirty_text.str.replace(r'>>', '')
# remove " "
dirty_text = dirty_text.str.replace(r' ', '')
# remove "\n"
dirty_text = dirty_text.str.replace(r'\n', '')
# remove "\r"
dirty_text = dirty_text.str.replace(r'\r', '')
# remove "\t"
dirty_text = dirty_text.str.replace(r'\t', '')
dirty_text = dirty_text.str.replace(r'​', '')
# remove text with only space
dirty_text = dirty_text.str.replace(r'^\s*$', '')
# remove url
# dirty_text = dirty_text.str.replace(r'[^ก-๙0-9]', '')
# removee url
dirty_text = dirty_text.str.replace(r'https?://\S+|www\.\S+', '')
# remove all number except number 5
dirty_text = dirty_text.str.replace(r'[0-46-9]', '')
# remove empty string
dirty_text = dirty_text[dirty_text != '']
dirty_text

1       ดูเหมือนเหตุผลหลักที่ผู้ไม่เห็นด้วยกับคุณพิธาใ...
2       ผมเห็นใจเพื่อไทยที่ต้องทนอยู่ใต้เงาของก้าวไกลท...
3             ฉันเกิดในรัฐธรรมนูญหน้าหี#มิตรสหายท่านหนึ่ง
4       ประชาธิปไตยที่พวกมึงยกย่องมันก็เกิดมาจากรัฐประ...
5       แดงมึงจะดีใจกับที่ส้มโดนอำนาจดำเล่นงานทำไมรัฐธ...
                              ...                        
4236    เมื่อก่อนก็เคยมีอากิบะตันไงเจ้าของเดียวกับโม่ง...
4237                    เว็บไรวะกุไม่รู้จักรุ่จักแต่trikx
4238                             โม่งคล้ายๆอยู่แต่แม่งกาก
4239                             วิศวกรท่านนึงบอกกูไว้อ่ะ
4240                                       มันคือปารีเสิน
Name: reply_text, Length: 4031, dtype: object

In [193]:
from attacut import tokenize, Tokenizer

In [194]:
%%time
tsentences = []
for st in dirty_text:
	tokens = tokenize(st.strip())
	tsentences.append(tokens)

tsentences[0:5]

CPU times: total: 2.73 s
Wall time: 42.4 s


[['ดู',
  'เหมือน',
  'เหตุผล',
  'หลัก',
  'ที่',
  'ผู้',
  'ไม่',
  'เห็น',
  'ด้วย',
  'กับ',
  'คุณพิธา',
  'ให้',
  'เป็น',
  'นายก',
  'คือ',
  'ข้อ',
  'เสนอ',
  'แก้',
  'มาตรา',
  'คำ',
  'ถาม',
  'คือ',
  'ทำไม',
  'จึงuntouchable',
  'และ',
  'ห้าม',
  'พูด',
  'ถึง',
  'เลย',
  'ใน',
  'สภา',
  'ทั้งที่',
  'ก็',
  'รู้',
  'อยู่',
  'ว่า',
  'มัน',
  'ก่อ',
  'ให้',
  'เกิด',
  'ปัญหา',
  'ถ้า',
  'ไม่',
  'เชื่อ',
  'ว่า',
  'ก่อ',
  'ให้',
  'เกิด',
  'ปัญหา',
  'ไม่',
  'เชื่อ',
  'ว่า',
  'ถูก',
  'ใช้',
  'เป็น',
  'เครื่องมือ',
  'ทาง',
  'การ',
  'เมือง',
  'ก็',
  'ลอง',
  'ไป',
  'คุย',
  'ใน',
  'สภา',
  'สิ',
  'ว่า',
  'ทำไม',
  'คิด',
  'อย่าง',
  'นั้น',
  'แล้ว',
  'convinceฝั่ง',
  'ที่',
  'เขา',
  'เชื่อ',
  'หรือ',
  'คน',
  'ที่',
  'เป็น',
  'กลาง',
  'ใน',
  'เรื่อง',
  'นี้',
  'จะ',
  'ได้',
  'มี',
  'พวก',
  'มาก',
  'ขึ้น',
  'ไม่',
  'ดี',
  'กว่า',
  'หรือ',
  'ครับ',
  'เรื่อง',
  'ที่',
  'สังคม',
  'มี',
  'ความ',
  'เห็น',
  'ต่าง',
  'กัน

In [195]:
%%time
dimension = 128
w2v_model = Word2Vec(tsentences, min_count=1, vector_size=dimension, workers=12, sg=1, epochs=200)

# Get the word embeddings
embeddings = w2v_model.wv.vectors
print(w2v_model)

Word2Vec<vocab=11177, vector_size=128, alpha=0.025>
CPU times: total: 33.4 s
Wall time: 18 s


In [168]:
# get top sim "อร่อย"
w2v_model.wv.most_similar('CGM', topn=20)

[('ไอดอลเกิร์ลกรุ๊ป', 0.8553664684295654),
 ('นครล้านนา', 0.7920939922332764),
 ('#>/girlg', 0.7647486329078674),
 ('ใหม่เเห่งเเดนเหนือriseofchampoo', 0.7609695792198181),
 ('เเชมเเชม>', 0.7593297362327576),
 ('ผูกพันนิรันดร์', 0.7566373944282532),
 ('องหงิญPrincessCGMทรงพระเจริญ', 0.7552969455718994),
 ('>/girlg', 0.7468482851982117),
 ('มู้CGM', 0.7330686450004578),
 ('นครล้านนาตอนณชรี', 0.7320119738578796),
 ('-ปี>', 0.7247273325920105),
 ('ทำไม#>', 0.7217938899993896),
 ('5-ปี>', 0.7127153873443604),
 ('รีทัช', 0.7010517120361328),
 ('//-ปี>', 0.7004674673080444),
 ('BNKไอดอลเกิร์ลกรุ๊ป', 0.6989429593086243),
 ('.>/girlg', 0.6985284090042114),
 ('ผูกพันนิรันดร์เเต่', 0.6913595199584961),
 ('(Vol.ละมู้', 0.6910515427589417),
 ('/girlg', 0.6793577671051025)]

In [196]:
import csv

words = list(w2v_model.wv.index_to_key)
vectors = list(w2v_model.wv.vectors)

with open('w2vG_10K_sex.tsv', 'w',  encoding='utf-8') as out_file:
	tsv_writer = csv.writer(out_file, delimiter='\t')
	for v in vectors:
		tsv_writer.writerow(v)

with open('labelG_10K_sex.tsv', 'w',  encoding='utf-8') as out_file:
	tsv_writer = csv.writer(out_file, delimiter='\t')
	for word in words:
		tsv_writer.writerow([word])